# Training driving model with behavioral cloning

## In this notebook, we will train a NN using samples obtained from Udacity Simulator

In [1]:
import os
import csv
import pdb

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Lambda
from keras.layers import Cropping2D
from keras.layers.convolutional import Convolution2D, Conv2D
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint


datafolder = '/media/josealb/HDD_1/Datasets/Self_driving/Simulator_Data/ND_Sample/'
#datafolder = '/media/josealb/HDD_1/Datasets/Self_driving/Simulator_Data/Track_1/'
samples = []
with open(datafolder+'driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name_center = datafolder+'IMG/'+batch_sample[0].split('/')[-1]
                name_left = datafolder+'IMG/'+batch_sample[1].split('/')[-1]
                name_right = datafolder+'IMG/'+batch_sample[2].split('/')[-1]

                #name = datafolder+'IMG/'+batch_sample[0].split('\\')[-1]

                center_image = cv2.imread(name_center)
                left_image = cv2.imread(name_left)
                right_image = cv2.imread(name_right)

                center_angle = float(batch_sample[3])
                center_angle = center_angle#Makes Neural network turn more aggresively
                correction = 0.2
                
                left_angle = center_angle + correction
                right_angle= center_angle - correction
                
                images.append(center_image) #should be made random
                angles.append(center_angle)
                
                images.append(left_image)
                angles.append(left_angle)
                
                images.append(right_image)
                angles.append(right_angle)                

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

#ch, row, col = 3, 80, 320  # Trimmed image format
ch, row, col = 3, 160, 320  # UnTrimmed image format
#ELU instead of RELU?

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 

model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(row,col,ch)))
model.add(Lambda(lambda x: x/127.5 - 1.))#,
       # input_shape=(row, col, ch),
        #output_shape=(row, col, ch)))
#model.add(Convolution2D(24,9,9, activation="elu"))
#model.add(BatchNormalization())
model.add(Conv2D(24,5,5,subsample=(2,2), activation="elu"))
model.add(Convolution2D(36,5,5,subsample=(2,2), activation="elu"))
#model.add(BatchNormalization())
model.add(Convolution2D(48,5,5,subsample=(2,2), activation="elu"))
#model.add(BatchNormalization())
model.add(Convolution2D(64,3,3, activation="elu"))
#model.add(BatchNormalization())
model.add(Convolution2D(64,3,3, activation="elu"))
##model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1164))
#model.add(BatchNormalization())
model.add(Dense(100))
#model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(50))
#model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Dropout(0.5))
#model.add(BatchNormalization())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

checkpoint = ModelCheckpoint('model-{epoch:03d}-{val_loss:03f}.h5',
                            monitor='val_loss',
                            verbose=0,
                            save_best_only=True,
                            mode = 'auto')

model.fit_generator(train_generator, samples_per_epoch= \
            len(train_samples)*3, validation_data=validation_generator, \
            nb_val_samples=len(validation_samples), nb_epoch=3, callbacks= [checkpoint], verbose=1)

model.save('model.h5')

Using TensorFlow backend.


Epoch 1/3
19284/19284 [==============================] - 26s - loss: 1.9723 - val_loss: 0.0333
Epoch 2/3
19284/19284 [==============================] - 23s - loss: 0.1941 - val_loss: 0.0303
Epoch 3/3
19284/19284 [==============================] - 23s - loss: 0.1202 - val_loss: 0.0335


In [2]:
model.fit_generator(train_generator, samples_per_epoch= \
            len(train_samples), validation_data=validation_generator, \
            nb_val_samples=len(validation_samples), nb_epoch=10, callbacks= [checkpoint], verbose=1)


Epoch 1/10
6428/6428 [==============================] - 14s - loss: 0.0446 - val_loss: 0.0402
Epoch 2/10
6428/6428 [==============================] - 14s - loss: 0.0429 - val_loss: 0.0391
Epoch 3/10
6428/6428 [==============================] - 16s - loss: 0.0431 - val_loss: 0.0389
Epoch 4/10
6428/6428 [==============================] - 16s - loss: 0.0420 - val_loss: 0.0389
Epoch 5/10
6428/6428 [==============================] - 16s - loss: 0.0418 - val_loss: 0.0382
Epoch 6/10
6428/6428 [==============================] - 17s - loss: 0.0408 - val_loss: 0.0383
Epoch 7/10
6428/6428 [==============================] - 16s - loss: 0.0402 - val_loss: 0.0387
Epoch 8/10
6428/6428 [==============================] - 16s - loss: 0.0409 - val_loss: 0.0380
Epoch 9/10
6428/6428 [==============================] - 15s - loss: 0.0401 - val_loss: 0.0370
Epoch 10/10
6428/6428 [==============================] - 14s - loss: 0.0388 - val_loss: 0.0375


In [1]:
train_samples

NameError: name 'train_samples' is not defined